In [ ]:
import laser_lib
import numpy as np
import math
import time
import cv2
from skimage.color import rgb2lab, lab2rgb

In [ ]:
queue = laser_lib.DacQueue()

In [ ]:
def signer(f, T, phase=0.0):
    '''
    Produces spaced out points from 0 to 1 whose distance between 
    consecutive points is sinusoidal 
    Input:  f - frequency 
            T - number of samples
            phase - phase shift (0, 1)
    Returns: x - positions on the line
             dx - the derivative of the function of x
    '''
    # Convert to radians
    f = f*2*np.pi
    # Generate the baseline 
    t = np.linspace(0.0+phase, 1.0+phase, T)
    # Find points 
    x = (f*t - np.cos(f*t) + 1)/f
    # Calculate derivative
    dx = (np.sin(f*t) + 1)/2
    
    return x, dx

def triangle(theta, alpha, sincos='sin'):
    '''
    Returns a wave between a sine wave and triangle wave depending on alpha
    alpha = 0 - sine wave
    alpha = 1 - triangle wave
    '''
    if(sincos == 'sin'):
        ps = 0
    elif(sincos == 'cos'):
        ps = np.pi/2
    return alpha*2*np.arcsin(np.sin(theta + ps))/np.pi + (1-alpha)*np.sin(theta + ps)
    
def ellipser(scale_x=1, scale_y=1, f_x=1, f_y=1, f_s=1000, phase_s=0, sintriang=0, colors=[(1,1,1)], T=500):
    '''
    Generates a pattern that is a function of ellipses 
    with parameters that can be interpolated between
    '''
    theta_norm, theta_dx = signer(f=f_s, phase=phase_s, T=T)
    theta = theta_norm*2*np.pi
    arr_pos = np.zeros((T, 2))
    arr_pos[:,0] = scale_x*triangle(f_x*theta, sintriang, 'sin')
    arr_pos[:,1] = scale_y*triangle(f_y*theta, sintriang, 'cos')
    
    # Color the pattern
    arr_col = np.ones((T, 3), np.float32)
    if(len(colors) == 1):
        # All one color
        arr_col[:, :] = colors[0][np.newaxis, :]
    if(len(colors) == 2):
        # Color based on speed of laser
        arr_col[theta_dx > 0.1, :] = colors[0]
        arr_col[theta_dx <= 0.1, :] = colors[1]
    
    return arr_pos, arr_col


### Beam pairs flashing back and forth

In [ ]:
T = 1000
theta = np.linspace(0, 2*np.pi, T)
freq = 10
arr_pos_l = np.zeros((T, 2))
arr_pos_r = np.zeros((T, 2))
arr_col_l = np.zeros((T, 3))
arr_col_r = np.zeros((T, 3))
scaler = 0.75
arr_pos[:, 0] = np.cos(freq*theta)
arr_pos_l[:, 0] = np.cos(freq*theta)*scaler + (1-scaler)
arr_pos_r[:, 0] = np.cos(freq*theta)*scaler - (1-scaler)
arr_col_l[arr_pos[:,0] >  0.99, :] = np.array([1,1,1])/6
arr_col_r[arr_pos[:,0] < -0.99, :] = np.array([1,1,1])/6
arr_col_l[arr_pos[:,0] < -0.99, :] = np.array([1,0,1])
arr_col_r[arr_pos[:,0] >  0.99, :] = np.array([1,0,1])

stay = 10
while(True):
    for i in range(stay):
        queue.submit(arr_pos_l, arr_col_l)
        
    for i in range(stay):
        queue.submit(arr_pos_r, arr_col_r)

### Concentric, Rotating, Discrete Beam Circles

In [ ]:
def rotate_pos(arr_pos, ang_rot):
    '''
    Rotates all points in arr_pos by ang_rot radians
    '''
    c, s = np.cos(ang_rot), np.sin(ang_rot)
    R = np.array(((c, -s), (s, c)))
    return np.matmul(arr_pos, R)

ang_rot = 0
T = 300
num_beams = 6
num_cols = 3
points_per_beam = T//num_beams
trip_col = np.array([[1,0,1],[0,0.5,1],[0,0,1]])
trip_col = np.tile(trip_col, (num_beams//num_cols, 1))
trip_col = np.repeat(trip_col, points_per_beam, axis=0)

while(True):
    ang_rot += 0.01
    arr_pos, arr_col = ellipser(scale_x=1, scale_y=1, f_x=1, f_y=1, f_s=num_beams, phase_s=0, sintriang=0, colors=[(0,0,0),(1,1,1)], T=T)
    queue.submit(rotate_pos(arr_pos, ang_rot), trip_col*arr_col)
    queue.submit(0.75*rotate_pos(arr_pos, -ang_rot), trip_col*arr_col)
    queue.submit(0.50*rotate_pos(arr_pos,  ang_rot), trip_col*arr_col)
    

In [ ]:
# ARC THAT EXPANDS
T = 500
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.ones((T, 3))

num_steps = 100
for i in range(num_steps//2):
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] < np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] > np.pi/2 - np.pi*i/num_steps)
    queue.submit(arr_pos, arr_col_out)
    
for i in range(num_steps//2):
    arr_col_out = arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] < np.pi/2 - np.pi*i/num_steps)
    queue.submit(arr_pos, arr_col_out)

In [ ]:
# Frames -> Patterns -> Sequences

In [ ]:
# ARC THAT EXPANDS
T = 500
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.ones((T, 3))

num_steps = 100
while(True):
    #Arc disappears from right and left side of circle (0)     
    for i in range(num_steps//2):
        arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] > np.pi*i/num_steps, theta[:, np.newaxis] < np.pi - np.pi*i/num_steps)
        queue.submit(arr_pos, arr_col_out)
        
        


In [ ]:
np.zeros((5, 3))[...,np.newaxis].shape

## Straight Up Villain (Chorus) 

In [ ]:
# This pattern is for the chorus, it is repeated throughout the song

# Time (seconds) between beats
Tb = 0.375
# Number of frames per beat
T = int(Tb*queue.dac_rate)

# 1) Beam fast move twice (2 beats)
# 2) Beam slow move once (2 beats)
# 3) Beams split

# test with just one beam 
p1 = np.array([-1, -1])
p2 = np.array([ 1, -1])

arr_pos = np.linspace(p1, p2, T)
arr_col = np.ones((T, 3))
print(arr_pos.shape, arr_col.shape)
queue.submit(arr_pos, arr_col)
    


In [ ]:
# Start off with a certain number of beams 
num_beams_init = 2
num_splits = 3
num_beams_end = num_beams_init**(num_splits+1)

axes = 1 # Symmetries, this increases w/ every split


In [ ]:
num_beams_end

In [ ]:
# 